In [49]:
import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from typing import List, Dict
import json

In [36]:
# === MODEL SETUP ===
MODEL_NAME = "intfloat/multilingual-e5-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).eval()
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, eleme

In [37]:
# === CONFIGURATION ===
TEXT_FOLDER = Path("npfl140/data/txt_extracted")
MAX_TOKENS = 256
STRIDE = 128
BATCH_SIZE = 8

In [39]:
def split_text_into_passages(text: str, max_tokens: int, stride: int) -> List[str]:
    token_ids = tokenizer.encode(text, add_special_tokens=False)
    # token_ids = tokenizer.encode(text, add_special_tokens=False)[:2048]

    passages = []
    for i in range(0, len(token_ids), stride):
        chunk = token_ids[i:i + max_tokens]
        if chunk:
            passages.append(tokenizer.decode(chunk, skip_special_tokens=True))
        if i + max_tokens >= len(token_ids):
            break
    return passages

In [41]:
@torch.no_grad()
def encode_passages(passages: List[str]) -> List[np.ndarray]:
    embeddings = []
    for i in range(0, len(passages), BATCH_SIZE):
        batch = [f"passage: {p}" for p in passages[i:i + BATCH_SIZE]]
        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)

        outputs = model(**inputs)
        hidden = outputs.last_hidden_state
        mask = inputs.attention_mask.unsqueeze(-1).float()
        masked = hidden * mask
        mean_pooled = masked.sum(1) / mask.sum(1)
        embeddings.extend(mean_pooled.cpu().numpy())
    return embeddings

In [55]:
def process_all_files(text_dir: Path) -> List[Dict]:
    result = []
    files = sorted(text_dir.glob("*.txt"))
    progress = tqdm(files, desc="Processing files")
    i = 0

    for file in progress:
        title = file.stem
        progress.set_postfix(file=title[:40])  # limit length to avoid overflow

        with open(file, "r", encoding="utf-8") as f:
            text = f.read()
        passages = split_text_into_passages(text, max_tokens=MAX_TOKENS, stride=STRIDE)
        if not passages:
            continue
        emb_vectors = encode_passages(passages)
        for passage, vec in zip(passages, emb_vectors):
            result.append({
                "title": title,
                "passage": passage,
                "embedding": vec.tolist()
            })
        # i += 1
        # if i == 3:
        #     break
    return result

In [56]:
OUTPUT_FILE = "npfl140/data/wiki_passages_with_embeddings.jsonl"

In [57]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for item in process_all_files(TEXT_FOLDER):
        # print(item["title"])
        f.write(json.dumps(item, ensure_ascii=False) + "\n")
        f.flush()

Processing files: 100%|██████████| 489/489 [05:38<00:00,  1.45it/s, file=Žítkovské_bohyně__kniha_]                


In [23]:
def process_single_file(file: Path) -> List[Dict]:
    title = file.stem
    with open(file, "r", encoding="utf-8") as f:
        text = f.read()
    passages = split_text_into_passages(text, max_tokens=MAX_TOKENS, stride=STRIDE)
    if not passages:
        return []

    emb_vectors = encode_passages(passages)
    return [
        {
            "title": title,
            "passage": passage,
            "embedding": vec.tolist()
        }
        for passage, vec in zip(passages, emb_vectors)
    ]

In [33]:
sample_file = Path("npfl140/data/txt_extracted/Disident.txt")
output = process_single_file(sample_file)

In [34]:
print(output)

[{'title': 'Disident', 'passage': 'Disident náhled|274x274pixelů|[[Václav Havel a Alexander Dubček 24. listopadu 1989 v pražské Laterně magice]] Disident [dysident] (lat., česky odpůrce, odštěpenec či odpadlík) je v politickém slova smyslu člověk odlišně smýšlející, odmítající oficiální vládnoucí ideologii, aktivní odpůrce establishmentu (zpravidla nenásilnou formou). Zejména v nedemokratických státech bývají disidenti za své přesvědčení pronásledováni a trestáni, a to i trestem odnětí svobody či trestem smrti.<br>Disent [dysent] je označení pro jednotlivce a skupiny, jež veřejně a otevřeně vyjadřují názory odlišné od oficiální vládnoucí ideologie. Etymologie Slovo „disident“ se začalo používat pro odpůrce totalitních režimů ve východním bloku v 60. letech 20. století ve Francii. Definice Podle Václava Bělohradského „disident v první řadě hájí to, co nesmíme obětovat v žádném politickém systému.', 'embedding': [0.013990922830998898, -0.890832781791687, 0.0789148136973381, -0.4982456564